# **Code to clean up the extracted Sponsor List**




# Load library

In [41]:
# Install requests
!pip install requests

# Install requests-ip-rotator
!pip3 install requests-ip-rotator

In [2]:
# Regular Python Data analyze library
import pandas as pd
import random

# Web related library
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup

# Progress Bar
from tqdm import tqdm

# Requests-ip-rotator to rotate IP
# Aim to avoid 429 HTTP error
# Reference: https://github.com/Ge0rg3/requests-ip-rotator
from requests_ip_rotator import ApiGateway

# Add request error handler
from requests.exceptions import HTTPError, ReadTimeout

# Modify code from original python library: googlesearch-python

original website: https://pypi.org/project/googlesearch-python/#description \
original code: https://github.com/Nv7-GitHub/googlesearch

In [92]:
import random

def get_useragent():
    """
    Generates a random user agent string mimicking the format of various software versions.

    The user agent string is composed of:
    - Lynx version: Lynx/x.y.z where x is 2-3, y is 8-9, and z is 0-2
    - libwww version: libwww-FM/x.y where x is 2-3 and y is 13-15
    - SSL-MM version: SSL-MM/x.y where x is 1-2 and y is 3-5
    - OpenSSL version: OpenSSL/x.y.z where x is 1-3, y is 0-4, and z is 0-9

    Returns:
        str: A randomly generated user agent string.
    """
    lynx_version = f"Lynx/{random.randint(2, 3)}.{random.randint(8, 9)}.{random.randint(0, 2)}"
    libwww_version = f"libwww-FM/{random.randint(2, 3)}.{random.randint(13, 15)}"
    ssl_mm_version = f"SSL-MM/{random.randint(1, 2)}.{random.randint(3, 5)}"
    openssl_version = f"OpenSSL/{random.randint(1, 3)}.{random.randint(0, 4)}.{random.randint(0, 9)}"
    return f"{lynx_version} {libwww_version} {ssl_mm_version} {openssl_version}"

In [93]:
"""googlesearch is a Python library for searching Google, easily."""
from time import sleep
from bs4 import BeautifulSoup
from requests import get
from urllib.parse import unquote # to decode the url


def _req(term, results, lang, start, proxies, timeout, safe, ssl_verify, region):
    resp = session.get(
        url="https://www.google.com/search",
        headers={
            "User-Agent": get_useragent(),
            "Accept": "*/*"
        },
        params={
            "q": term,
            "num": results + 2,  # Prevents multiple requests
            "hl": lang,
            "start": start,
            "safe": safe,
            "gl": region,
        },
        proxies=proxies,
        timeout=timeout,
        verify=ssl_verify,
        cookies = {
            'CONSENT': 'PENDING+987', # Bypasses the consent page
            'SOCS': 'CAESHAgBEhIaAB',
        }
    )
    resp.raise_for_status()
    return resp


class SearchResult:
    def __init__(self, url, title, description):
        self.url = url
        self.title = title
        self.description = description

    def __repr__(self):
        return f"SearchResult(url={self.url}, title={self.title}, description={self.description})"


def search(term, num_results=10, lang="en", proxy=None, advanced=False, sleep_interval=0, timeout=5, safe="active", ssl_verify=None, region=None, start_num=0, unique=False):
    """Search the Google search engine"""

    # Proxy setup
    proxies = {"https": proxy, "http": proxy} if proxy and (proxy.startswith("https") or proxy.startswith("http") or proxy.startswith("socks5")) else None

    start = start_num
    fetched_results = 0  # Keep track of the total fetched results
    fetched_links = set() # to keep track of links that are already seen previously

    while fetched_results < num_results:
        # Send request
        resp = _req(term, num_results - start,
                    lang, start, proxies, timeout, safe, ssl_verify, region)

        # put in file - comment for debugging purpose
        # with open('google.html', 'w') as f:
        #     f.write(resp.text)

        # Parse
        soup = BeautifulSoup(resp.text, "html.parser")
        result_block = soup.find_all("div", class_="ezO2md")
        new_results = 0  # Keep track of new results in this iteration

        for result in result_block:
            # Find the link tag within the result block
            link_tag = result.find("a", href=True)
            # Find the title tag within the link tag
            title_tag = link_tag.find("span", class_="CVA68e") if link_tag else None
            # Find the description tag within the result block
            description_tag = result.find("span", class_="FrIlee")

            # Check if all necessary tags are found
            if link_tag and title_tag and description_tag:
                # Extract and decode the link URL
                link = unquote(link_tag["href"].split("&")[0].replace("/url?q=", "")) if link_tag else ""
            # Extract and decode the link URL
            link = unquote(link_tag["href"].split("&")[0].replace("/url?q=", "")) if link_tag else ""
            # Check if the link has already been fetched and if unique results are required
            if link in fetched_links and unique:
                continue  # Skip this result if the link is not unique
            # Add the link to the set of fetched links
            fetched_links.add(link)
            # Extract the title text
            title = title_tag.text if title_tag else ""
            # Extract the description text
            description = description_tag.text if description_tag else ""
            # Increment the count of fetched results
            fetched_results += 1
            # Increment the count of new results in this iteration
            new_results += 1
            # Yield the result based on the advanced flag
            if advanced:
                yield SearchResult(link, title, description)  # Yield a SearchResult object
            else:
                yield link  # Yield only the link

            if fetched_results >= num_results:
                break  # Stop if we have fetched the desired number of results

        if new_results == 0:
            #If you want to have printed to your screen that the desired amount of queries can not been fulfilled, uncomment the line below:
            #print(f"Only {fetched_results} results found for query requiring {num_results} results. Moving on to the next query.")
            break  # Break the loop if no new results were found in this iteration

        start += 10  # Prepare for the next set of results
        sleep(sleep_interval)

# Sort the Sponsor list and perform manual data integration

In [ ]:
# Load CSV into a Data Pool DataFrame
# Change encoding from 'utf-8' to 'latin1' to avoid invalid continuation byte
# df = pd.read_csv('BOC_update - Sheet1.csv', encoding='latin1')
df = pd.read_csv('')

# Apply .strip() to all string values in the DataFrame using apply with a lambda function
df = df.apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))

# Now all string values in df_pool will have leading/trailing spaces removed
print("Stripped all leading/trailing spaces from string attributes.")

Stripped all leading/trailing spaces from string attributes.


In [ ]:
df

In [8]:
# Sort the Pandas Dataframe by the sponsor column
df = df.sort_values(by='Sponsors', ascending=True)
df = df.reset_index(drop=True)

In [ ]:
df

Extract the sorted csv, and perform manual data integration

In [ ]:
# Export to csv file
df.to_csv('', index=False)

# Loop through Sponsor List, and systematically extract the homepage url for each sponsor

At the point, Sponsor list should be integrated

In [ ]:
# Load CSV into a Data Pool DataFrame
# Change encoding from 'utf-8' to 'latin1' to avoid invalid continuation byte
df = pd.read_csv('')

## Further Pre-processing

In [13]:
# Cleaning function for Sponsor name
def clean_sponsor_name(sponsor):
  # Make a copy of sponsor's name
  sponsor_short_name = sponsor

  # Remove specific endings
  if sponsor_short_name.endswith(", Inc."):
    sponsor_short_name = sponsor_short_name[:-6]
  if sponsor_short_name.endswith(" Inc."):
    sponsor_short_name = sponsor_short_name[:-5]
  if sponsor_short_name.endswith(", Inc"):
    sponsor_short_name = sponsor_short_name[:-5]
  if sponsor_short_name.endswith(" Inc"):
    sponsor_short_name = sponsor_short_name[:-4]
  if sponsor_short_name.endswith(", LLC"):
    sponsor_short_name = sponsor_short_name[:-5]
  if sponsor_short_name.endswith(" LLC"):
    sponsor_short_name = sponsor_short_name[:-4]
  if sponsor_short_name.endswith(", LLP"):
    sponsor_short_name = sponsor_short_name[:-5]
  if sponsor_short_name.endswith(" LLP"):
    sponsor_short_name = sponsor_short_name[:-4]
  if sponsor_short_name.endswith(" Co."):
    sponsor_short_name = sponsor_short_name[:-4]
  if sponsor_short_name.endswith(", In"):
    sponsor_short_name = sponsor_short_name[:-4]

  return sponsor_short_name.strip()

In [14]:
# Apply the function to the "Sponsors" column
df['Sponsors'] = df['Sponsors'].apply(clean_sponsor_name)

In [ ]:
df

## Google search each sponsor

AWS: https://aws.amazon.com/api-gateway/

In [ ]:
# Create gateway object and initialise in AWS
gateway = ApiGateway("https://www.google.com",
                     access_key_id="",
                     access_key_secret="")
gateway.start()

# Assign gateway to session
session = requests.Session()
session.mount("https://www.google.com", gateway)

Starting API gateways in 10 regions.
Using 10 endpoints with name 'https://www.google.com - IP Rotate API' (10 new).


In [110]:
# Create Sponsor list
Sponsor_list = df['Sponsors'].tolist()
# Create Homepage URL list
Sponsor_URL_list = []
# Create Homepage Domain list
Sponsor_Domain_list = []
# Create Sponsor Shortname list
Sponsor_Shortname_list = []

In [111]:
# define number of retries
retries = 5

# Find the homepage for each Sponsor
for sponsor in tqdm(Sponsor_list):
  # To track progress
  print("Working on:", sponsor)

  # Call google search on each sponsor
  for attempt in range(retries):
    try:
      sponsor_url_candidates = list(search(sponsor, num_results=5))
      break
    # Immediate retry after detecting 429 or Readtime Error
    except HTTPError as http_err:
      if http_err.response.status_code == 429:
        print("429 Too Many Requests - will restart soon")
      else:
        raise http_err
    except ReadTimeout:
        print(f"ReadTimeout detected on attempt {attempt + 1}, retrying...")

  # Extract main page url, which is the highest ranked (first) returning result
  if len(sponsor_url_candidates) > 0:
    # Extract main page URL
    # Assume the first returning result from google is home url for the sponsor
    # We will take the first valid url
    for url in sponsor_url_candidates:
      if url and not url.startswith('/'):
        sponsor_url = url
        break
    else:
      # Very unlikely we will get non-valid url
      raise ValueError("No valid sponsor URL found.")

    # Extract shortname from this url
    # Remove all front and rear information
    # We will actually keep the ".com" etc, to reduce ambiguity (ex. mn vs. mn.gov)
    sponsor_domain = sponsor_url.split('/')[2]

    shortname = sponsor_domain
    # If start with "www.", remove that as well
    if shortname.startswith("www."):
      shortname = shortname[4:]
    # Check if shortname is one of these types:
    if shortname == "facebook.com" or shortname == "twitter.com" or shortname == "instagram.com" or shortname == "nba.com" or shortname == "mlb.com" or shortname == "nhl.com":
      # adopt to different format
      shortname = sponsor_url.split('/')[3]
      sponsor_domain = sponsor_domain + '/' + shortname
    if shortname == "linkedin.com" or shortname == "reddit.com" or shortname == "forbes.com":
      shortname = sponsor_url.split('/')[4]
      sponsor_domain = sponsor_domain + '/' + sponsor_url.split('/')[3] + '/' + shortname

    # Remove ".com" etc in the end
    if shortname.endswith(".com") or shortname.endswith(".org") or shortname.endswith(".net") or shortname.endswith(".edu") or shortname.endswith(".gov") or shortname.endswith(".vc"):
      shortname = shortname[:-4]

    # Append to list
    Sponsor_URL_list.append(sponsor_url)
    Sponsor_Domain_list.append(sponsor_domain)
    Sponsor_Shortname_list.append(shortname)

  else:
    # No homepage url found
    # Append to list
    Sponsor_URL_list.append("None")
    Sponsor_Domain_list.append("None")
    Sponsor_Shortname_list.append("None")

  0%|          | 0/14 [00:00<?, ?it/s]

Working on: Access to Capital for Entrepreneurs


  7%|▋         | 1/14 [00:00<00:08,  1.46it/s]

Working on: Advanced System Development


 14%|█▍        | 2/14 [00:01<00:08,  1.41it/s]

...

 86%|████████▌ | 12/14 [00:10<00:01,  1.19it/s]

Working on: University of Georgia Small Business Development Center


 93%|█████████▎| 13/14 [00:10<00:00,  1.43it/s]

Working on: bip ventures


100%|██████████| 14/14 [00:11<00:00,  1.25it/s]


In [112]:
# Check results
print("Length of Sponsor_list:", len(Sponsor_list))
print("Length of Sponsor_urls:", len(Sponsor_URL_list))
print("Length of Sponsor_url_shortnames:", len(Sponsor_Domain_list))
print("Length of Sponsor_shortnames:", len(Sponsor_Shortname_list))

Length of Sponsor_list: 14
Length of Sponsor_urls: 14
Length of Sponsor_url_shortnames: 14
Length of Sponsor_shortnames: 14


In [113]:
# Convert List to column in dataframe
df["Sponsor_URL"] = Sponsor_URL_list
df["Sponsor_Domain"] = Sponsor_Domain_list
df["Sponsor_Shortname"] = Sponsor_Shortname_list

In [114]:
gateway.shutdown()

Deleting gateways for site 'https://www.google.com'.
Deleted 10 endpoints with for site 'https://www.google.com'.


['bmhmyr8j91',
 'z7cc6delpd',
 '3eg43f6cz4',
 '1a64g5tml0',
 'qm8w1ykwck',
 '2por5eobs0',
 'mctt6pllcg',
 '2c5b60s6ce',
 'sr4ak718z8',
 'wrr99tniuj']

In [ ]:
df

# Final Check

Make necessary modifications

## Modifier

In [122]:
modify_index = 0

In [ ]:
df.loc[modify_index, "Sponsor_URL"] = "None"

In [ ]:
df.loc[modify_index, "Sponsor_Domain"] = ""

In [123]:
df.loc[modify_index, "Sponsor_Shortname"] = "ace loans"

## Full Table Display

In [ ]:
# Set the option to display all rows
pd.set_option('display.max_rows', None)

In [ ]:
df

In [ ]:
pd.reset_option('display.max_rows')

In [ ]:
# Export to csv file
df.to_csv('', index=False)